In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage.measure import block_reduce
import pickle

from scipy.stats import skew, kurtosis
from sklearn.linear_model import LinearRegression
from scipy.ndimage.filters import gaussian_filter as gf

from morans import morans

In [5]:
# Create a square 2D Ising Lattice 
# Initial magnetization state of the elements in the lattice is uniform (u) or randomized (r)

class IsingLattice:

    def __init__(self, initial_state, size, J, mask = None, h=0):
        self.size = size # size of each dimension
        

        if J is np.ndarray:
            self.J = J
        else:
            self.J = J*np.ones((size,size))
        self.h = h*np.ones((self.size,self.size)) # array of (size,size)
        
        self.system = self._build_system(initial_state)

        if mask is None:
            self.mask = np.ones((size,size))
        else:
            self.mask = mask

        # self.T = temperature*np.ones((self.size,self.size)) # array of (size,size)
        # self.k = 1.38 * (10**(-23))

    @property
    def sqr_size(self):
        return (self.size, self.size)

    def _build_system(self, initial_state):
        """Build the system

        Build either a randomly distributed system or a homogeneous system (for
        watching the deterioration of magnetization

        Parameters
        ----------
        initial_state : str: "r" for random or "u" for uniform (all +1)
            Initial state of the lattice. 
        """

        if initial_state == 'r':
            system = np.random.choice([-1, 1], self.sqr_size)
        elif initial_state == 'u':
            system = np.ones(self.sqr_size)
        else:
            raise ValueError(
                "Initial State must be 'r', random, or 'u', uniform"
            )

        return system
    
    def _bc(self, i):
        """Apply periodic boundary condition

        Check if a lattice site coordinate falls out of bounds. If it does,
        apply periodic boundary condition

        Assumes lattice is square

        Parameters
        ----------
        i : int
            lattice site coordinate

        Return
        ------
        int
            corrected lattice site coordinate
        """
        if i >= self.size:
            return 0
        if i < 0:
            return self.size - 1
        else:
            return i

    def energy(self, N, M):
        """Calculate the energy of spin interaction at a given lattice site
        i.e. the interaction of a Spin at lattice site n,m with its 4 neighbors

        - S_n,m*(S_n+1,m + Sn-1,m + S_n,m-1, + S_n,m+1)

        Parameters
        ----------
        N : int
            lattice site coordinate
        M : int
            lattice site coordinate

        Return
        ------
        float
            energy of the site
        """
        interactions = -self.system[N, M]*self.J[N,M]*(self.system[self._bc(N - 1), M] + self.system[self._bc(N + 1), M]+ self.system[N, self._bc(M - 1)] + self.system[N, self._bc(M + 1)])
        external = -self.system[N, M]*self.h[N,M]
        energy = interactions + external
        return energy
    
    @property
    def internal_energy(self):
        e = 0
        E = 0
        E_2 = 0

        for i in range(self.size):
            for j in range(self.size):
                e = self.energy(i, j)
                E += e
                E_2 += e**2

        U = (1./self.size**2)*E
        U_2 = (1./self.size**2)*E_2

        return U, U_2

    @property
    def heat_capacity(self,temp):
        U, U_2 = self.internal_energy
        return np.mean((U_2 - U**2)/np.power(temp,2))

    @property
    def magnetization(self):
        """Find the overall magnetization of the system
        """
        return np.sum(self.system)/self.size**2   #Maybe get rid of abs or add it
    
def hc(lattice,temp):
    U, U_2 = lattice.internal_energy
    return np.mean((U_2 - U**2)/np.power(temp,2))

def create_params():
    try:
        rng = np.random.default_rng()
    except AttributeError:
        rng = np.random.RandomState()


    J_mean = 2**(5*rng.uniform())  # why 5? can choose something else
    J_std = 0.3*rng.uniform()*J_mean # 0 - 30% of mean

    #J = J_mean*np.ones((size,size)) + J_std*(np.random.randn(size,size))

    Tc = 2*J_mean/(np.log(1+np.sqrt(2))) # critical temperature

    null = rng.choice([0,1])

    offset_dir =rng.choice([-1,1])
    spatial_coarse_graining = rng.choice(np.arange(3,8))
    #temporal_coarse_graining = rng.choice(np.arange(3,8))
    #spatial_coarse_graining = 1
    temporal_coarse_graining = 1
    epoch_len = 5000

    if null: # No Transition run (does not go through Tc)
        Tb1 = Tc * (0.2 + 0.2*rng.uniform())
        Tb2 = Tc * (0.4 + 0.2*rng.uniform())
            
        Tbounds = Tc + offset_dir*np.array([Tb1,Tb2])
        Tbounds = rng.permutation(Tbounds)

    else: # Transition Run (goes through Tc)
        Tb1 = Tc * (0.7 - 0.4*rng.uniform())
        Tb2 = Tc * (1.2 + 0.4*rng.uniform())
        #Tbounds = np.sort(np.array([Tb1,Tb2]))[::-1] # descending order, why ?? *******************
        Tbounds = rng.choice(([[Tb1,Tc],[Tc,Tb2]]))
        Tbounds = rng.permutation(Tbounds)
    
    target_duration = rng.choice(list(range(350,450)))


    run_params = {'J_mean':J_mean, 'J_std':J_std, 'Tc':Tc, 
                    'spatial_coarse_graining':spatial_coarse_graining,
                    'temporal_coarse_graining':temporal_coarse_graining,
                    'epoch_len':epoch_len, 'null':null,'Tbounds':Tbounds,'target_duration':target_duration}
    

    return run_params



def run(lattice, temps,burn_time , epoch_len = 5000):
    """Run the simulation
    """
    # System = []
    # Magnetization = []
    # Heat_Capacity = []
    epochs = temps.shape[0]

    System = np.zeros((epochs,lattice.system.shape[0],lattice.system.shape[1]))
    Magnetization = np.zeros(epochs)
    Heat_capacity = np.zeros(epochs)

    for epoch,temp in enumerate(temps):    

        step_avg = np.zeros((lattice.size,lattice.size)) # why are we taking step average

        for step in range(epoch_len):
            # Randomly select a site on the lattice
            N, M = np.random.randint(0, lattice.size, 2)

            # Calculate energy of a flipped spin
            dE = -2*lattice.energy(N, M)

            # "Roll the dice" to see if the spin is flipped
            if dE <= 0.:
                lattice.system[N, M] *= -1
            elif np.exp(-dE/(temp)) > np.random.rand():
                lattice.system[N, M] *= -1

            step_avg += lattice.system

        step_avg = step_avg/epoch_len

        # check and account for burn time (write an if statement)
        System[epoch,:,:] = step_avg
        Magnetization[epoch] = lattice.magnetization
        Heat_capacity[epoch] = hc(lattice,temp)

        #System.append(step_avg)
        #Magnetization.append(lattice.magnetization) #should we not take average magnetization?
        #Heat_Capacity.append(lattice.heat_capacity(temp=temp))

        output= {"System":System,"Magnetization":Magnetization,"Heat_capacity":Heat_capacity}

    return output



def ising_run(temps, size, J, burn_time, epoch_len, initial_state):
    
    
    lattice = IsingLattice(initial_state=initial_state, size=size, J=J)
    out_vars = run(lattice=lattice, temps=temps, burn_time=burn_time, epoch_len=epoch_len)

    return out_vars
            

In [6]:
n_runs = 15
burn_time = 50
target_size = 9
#target_duration = 400

Result=[]


for r in range(n_runs):

    print(f"Run {r}")
    params = create_params()
    J_mean = params["J_mean"]
    J_std = params["J_std"]
    Tc = params["Tc"]
    spatial_coarse_graining = params["spatial_coarse_graining"]
    temporal_coarse_graining = params["temporal_coarse_graining"]
    epoch_len = params["epoch_len"]
    null = params["null"]
    Tbounds = params["Tbounds"]
    target_duration = params["target_duration"]

    sim_size = target_size*spatial_coarse_graining
    sim_duration = target_duration*temporal_coarse_graining
    sim_burn_time = burn_time*temporal_coarse_graining
    temps = np.linspace(Tbounds[0],Tbounds[1],sim_duration-(null*100))

    if temps[0] > Tc:
        initial_state = 'r'
    else:
        initial_state = 'u'


    if not null:
        if (Tbounds[0]>=Tc and Tbounds[0]>=Tc):
            temps = temps[-50::]
            initial_state = 'r'
        else:
            temps = temps[:-50:]
            initial_state = 'u'



    J = J_mean*np.ones((sim_size,sim_size)) + J_std*(np.random.randn(sim_size,sim_size))
    
    
    output = ising_run(temps,sim_size,J,burn_time,epoch_len,initial_state)

    System = output["System"]
    Magnetization = output["Magnetization"]
    Heat_capacity = output["Heat_capacity"]

    System_cg = block_reduce(System,block_size=(temporal_coarse_graining,spatial_coarse_graining,spatial_coarse_graining),func=np.nanmean)

    output["System_cg"]=System_cg
    output["null"]=null

    Result.append(output)
    


Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Run 10
Run 11
Run 12
Run 13
Run 14


In [7]:
with open("Train_Data_11","wb") as file:
    pickle.dump(Result,file)